# Complete Guide: Load OpenAI GPT-OSS-20B Model and Run Attacks

This notebook will:
1. Load the model properly
2. Test that it's working
3. Run the attack scenarios

## Step 1: Check GPU Status

In [ ]:
import torch

if not torch.cuda.is_available():
    print("WARNING: GPU NOT ENABLED!")
    print("Please go to: Runtime → Change runtime type → GPU → Save")
    print("Then run this cell again")
else:
    print(f"GPU Enabled: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    !nvidia-smi

## Step 2: Clone Repository and Setup

In [ ]:
import os
os.chdir('/content')

# Clone repository
!rm -rf openai-
!git clone https://github.com/Unknown1502/openai-.git openai-
%cd openai-
!pwd

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch
!pip install -q transformers>=4.45.0
!pip install -q accelerate>=0.25.0
!pip install -q bitsandbytes>=0.41.3
!pip install -q sentencepiece protobuf
!pip install -q aiohttp nest-asyncio tqdm numpy pandas

# For MXFP4 support
!pip install -q triton>=3.4.0
!pip install -q kernels

print("All dependencies installed")

## Step 4: Fix the Code

In [ ]:
# Apply necessary fixes
!sed -i 's/ENHANCED_COMPETITION_ATTACKS = ENHANCED_COMPETITION_ATTACK_VECTORS/# Already defined above/' src/competition/enhanced_attack_vectors.py
print("Code fixes applied")

## Step 5: Load the Model (Most Important Step)

In [ ]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer

# Clear GPU memory
gc.collect()
torch.cuda.empty_cache()

# Set environment for better memory management
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

model_name = "openai/gpt-oss-20b"

print("Loading model (this will take 5-10 minutes)...")

# Load tokenizer first
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer loaded")

# Try to load model with MXFP4 dequantization
print("\nLoading model with dequantization...")
print("This will download ~14GB and may take several minutes...")

try:
    # Force dequantization for MXFP4
    os.environ['MXFP4_DEQUANTIZE'] = '1'
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        offload_folder="offload",  # Use CPU offloading if needed
        offload_state_dict=True
    )
    print("\nModel loaded successfully!")
    print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.1f}B parameters")
    
except Exception as e:
    print(f"\nError loading model: {str(e)[:500]}")
    print("\nTrying alternative loading method...")
    
    # Alternative: Load without quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    print("Model loaded with alternative method")

## Step 6: Test the Model

In [ ]:
# Test that the model is working
print("Testing model...")

test_prompt = "Hello, I am an AI assistant. How can I help you today?"

# Tokenize
inputs = tokenizer(test_prompt, return_tensors="pt")
inputs = {k: v.cuda() for k, v in inputs.items()}

# Generate
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=100,
        temperature=0.7,
        do_sample=True,
        top_p=0.95
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nPrompt: {test_prompt}")
print(f"Response: {response}")
print("\nModel is working correctly!")

## Step 7: Save Model to Globals (For Competition Scripts)

In [ ]:
# Save model and tokenizer globally so competition scripts can use them
import sys

# Create a module to hold the model
class ModelHolder:
    model = model
    tokenizer = tokenizer

sys.modules['model_holder'] = ModelHolder

print("Model saved to global scope")
print("The competition scripts can now access the loaded model")

## Step 8: Run Competition (Quick Test)

In [ ]:
# Run a quick test with the enhanced competition runner
!python run_enhanced_competition.py --mode quick --strategy standard

## Step 9: Run Full Competition

In [ ]:
# Run the standard competition
!python run_competition.py

## Alternative: Run with Custom Model Loading

If the above doesn't work, use this custom runner:

In [ ]:
# Direct attack test with loaded model
from src.competition.attack_vectors import COMPETITION_ATTACK_VECTORS

print("Running direct attack test...")

# Test first attack
attack = COMPETITION_ATTACK_VECTORS[0]
print(f"\nTesting: {attack.name}")
print(f"Category: {attack.category}")

# Run the attack
inputs = tokenizer(attack.prompt_template, return_tensors="pt")
inputs = {k: v.cuda() for k, v in inputs.items()}

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=200,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nModel Response: {response}")

# Check for vulnerability
if any(keyword in response.lower() for keyword in ['yes', 'admit', 'true', 'correct']):
    print("\nPotential vulnerability detected!")
else:
    print("\nNo vulnerability detected")

## Troubleshooting

If you encounter issues:

1. **CUDA Out of Memory**: Restart runtime and try again
2. **Model won't load**: Ensure you have GPU enabled (A100 preferred)
3. **Import errors**: Re-run the dependency installation cell
4. **No vulnerabilities detected**: The model might be too safe - try different prompts